In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys,warnings,random
from tqdm import tqdm

import rasterio as rio
from rasterio import merge
from rasterio.windows import Window,from_bounds
from rasterio.plot import reshape_as_raster,reshape_as_image
import geopandas as gpd
from shapely.geometry import box, Point

import tensorflow as tf
from tensorflow import keras

In [95]:
AOI_PATH = './DATA/test_inference_AOI.geojson'
INPUT_PATH = "./processed_DATA/features.tif"
MODEL_PATH = './model/v2/weights-improvement-14-0.16.hdf5'

In [96]:
AOI = gpd.read_file(AOI_PATH)

In [97]:
AOI.crs

<Derived Projected CRS: EPSG:3067>
Name: ETRS89 / TM35FIN(E,N)
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Finland - onshore and offshore.
- bounds: (19.08, 58.84, 31.59, 70.09)
Coordinate Operation:
- name: TM35FIN
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [98]:
minx,miny,maxx,maxy = AOI.bounds.values.tolist()[0]
PX_size = 10
output_size = 48 * PX_size
input_size = 64 * PX_size
delta = input_size - output_size

In [99]:
x = np.arange(np.floor(minx), np.floor(maxx), output_size)
y = np.arange(np.floor(miny), np.floor(maxy), output_size)
xv, yv = np.meshgrid(x, y)
Left_Corner = gpd.GeoSeries.from_xy(xv.flatten(),yv.flatten())

In [100]:
grid_geom = Left_Corner.apply(lambda p: box(p.x - delta/2, p.y - delta/2, p.x + output_size + delta/2, p.y + output_size + delta/2))

In [101]:
best_model = keras.models.load_model(MODEL_PATH,compile=False)

In [102]:
for i in tqdm(range(len(grid_geom))):
    INPUT_IMG = rio.open(INPUT_PATH)
    minx,miny,maxx,maxy = grid_geom[i].bounds
    window = from_bounds(minx, miny, maxx, maxy, transform=INPUT_IMG.transform)
    transform = rio.transform.from_bounds(minx + delta/2, miny + delta/2, maxx - delta/2, maxy - delta/2, output_size, output_size)
    input_result = INPUT_IMG.read(window=window)
    pred_result = best_model.predict(reshape_as_image(input_result).reshape(1,64,64,9))

    OUTPUT_PATH = f'./processed_DATA/inferrence_patches_test/{minx}_{miny}_inf.tif'
    with rio.open(OUTPUT_PATH, "w",
               driver='GTiff',
               count=1,
               transform = transform,
               width=output_size,
               height=output_size,
               dtype=pred_result.dtype,
               crs="epsg:3067",) as output_file:
        output_file.write(pred_result[...,0])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1020/1020 [01:11<00:00, 14.32it/s]


In [103]:
items = os.listdir('./processed_DATA/inferrence_patches_test/')
img_list  = [names for names in items if names.endswith(".tif")]
images = [rio.open('./processed_DATA/inferrence_patches_test/'+fname) for fname in img_list]
full_image, transform = rio.merge.merge(images)

In [104]:
MOSAIC_PATH = './processed_DATA/inference_test_mosaic.tif'

In [105]:
with rio.open(MOSAIC_PATH, "w",
                   driver='GTiff',
                   count=full_image.shape[0],
                   transform = transform,
                   width=full_image.shape[2],
                   height=full_image.shape[1],
                   dtype=full_image.dtype,
                   crs="epsg:3067",) as output_file:
        output_file.write(full_image)